# ಉತ್ಪಾದನೆಯಲ್ಲಿ AutoGen ಏಜೆಂಟ್‌ಗಳು: ಗಮನಾರ್ಹತೆ ಮತ್ತು ಮೌಲ್ಯಮಾಪನ

ಈ ಪಾಠದಲ್ಲಿ, ನಾವು **[Autogen agents](https://github.com/microsoft/autogen)**-ನ ಆಂತರಿಕ ಹಂತಗಳನ್ನು (ಟ್ರೇಸ್‌ಗಳನ್ನು) **ನಿರೀಕ್ಷಿಸುವುದು** ಮತ್ತು [Langfuse](https://langfuse.com) ಬಳಸಿ ಅದರ ಕಾರ್ಯಕ್ಷಮತೆಯನ್ನು **ಮೌಲ್ಯಮಾಪನ ಮಾಡುವುದನ್ನು** ಕಲಿಯುತ್ತೇವೆ.

ಈ ಮಾರ್ಗದರ್ಶಿ ತಂಡಗಳು ಏಜೆಂಟ್‌ಗಳನ್ನು ವೇಗವಾಗಿ ಮತ್ತು ವಿಶ್ವಾಸಾರ್ಹವಾಗಿ ಉತ್ಪಾದನೆಗೆ ತರುವ **ಆನ್‌ಲೈನ್** ಮತ್ತು **ಆಫ್‌ಲೈನ್** ಮೌಲ್ಯಮಾಪನ ಮೆಟ್ರಿಕ್‌ಗಳನ್ನು ಒಳಗೊಂಡಿದೆ.

**ಏಕೆ AI ಏಜೆಂಟ್ ಮೌಲ್ಯಮಾಪನ ಮುಖ್ಯವಾಗಿದೆ:**
- ಕಾರ್ಯಗಳು ವಿಫಲವಾದಾಗ ಅಥವಾ ಅಲ್ಪಮಟ್ಟದ ಫಲಿತಾಂಶಗಳನ್ನು ನೀಡಿದಾಗ ಸಮಸ್ಯೆಗಳನ್ನು ಡೀಬಗ್ ಮಾಡುವುದು
- ನಿಖರ ಸಮಯದಲ್ಲಿ ವೆಚ್ಚ ಮತ್ತು ಕಾರ್ಯಕ್ಷಮತೆಯನ್ನು ಗಮನಿಸುವುದು
- ನಿರಂತರ ಪ್ರತಿಕ್ರಿಯೆಯ ಮೂಲಕ ವಿಶ್ವಾಸಾರ್ಹತೆ ಮತ್ತು ಸುರಕ್ಷತೆಯನ್ನು ಸುಧಾರಿಸುವುದು


## ಹಂತ 1: ಪರಿಸರ ವ್ಯತ್ಯಾಸಗಳನ್ನು ಹೊಂದಿಸಿ

[Langfuse Cloud](https://cloud.langfuse.com/) ಗೆ ಸೈನ್ ಅಪ್ ಮಾಡುವ ಮೂಲಕ ಅಥವಾ [Langfuse ಅನ್ನು ಸ್ವಯಂ-ಹೋಸ್ಟ್ ಮಾಡುವುದು](https://langfuse.com/self-hosting) ಮೂಲಕ ನಿಮ್ಮ Langfuse API ಕೀಗಳನ್ನು ಪಡೆಯಿರಿ.

_**ಗಮನಿಸಿ:** ಸ್ವಯಂ-ಹೋಸ್ಟ್ ಮಾಡುವವರು [Terraform modules](https://langfuse.com/self-hosting/azure) ಅನ್ನು ಬಳಸಿಕೊಂಡು Azure ನಲ್ಲಿ Langfuse ಅನ್ನು ನಿಯೋಜಿಸಬಹುದು. ಬದಲಾಗಿ, ನೀವು [Helm chart](https://langfuse.com/self-hosting/kubernetes-helm) ಬಳಸಿ Kubernetes ನಲ್ಲಿ Langfuse ಅನ್ನು ನಿಯೋಜಿಸಬಹುದು._


In [5]:
import os

# Get keys for your project from the project settings page: https://cloud.langfuse.com
os.environ["LANGFUSE_PUBLIC_KEY"] = "pk-lf-..." 
os.environ["LANGFUSE_SECRET_KEY"] = "sk-lf-..." 
os.environ["LANGFUSE_HOST"] = "https://cloud.langfuse.com" # 🇪🇺 EU region
# os.environ["LANGFUSE_HOST"] = "https://us.cloud.langfuse.com" # 🇺🇸 US region

In [1]:
!pip install langfuse openlit


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


ಪರಿಸರ ಚರಾಂಶಗಳನ್ನು ಹೊಂದಿಸಿದ ನಂತರ, ನಾವು ಈಗ ಲಾಂಗ್‌ಫ್ಯೂಸ್ ಕ್ಲೈಂಟ್ ಅನ್ನು ಪ್ರಾರಂಭಿಸಬಹುದು. `get_client()` ಪರಿಸರ ಚರಾಂಶಗಳಲ್ಲಿ ನೀಡಲಾದ ಪ್ರಮಾಣಪತ್ರಗಳನ್ನು ಬಳಸಿಕೊಂಡು ಲಾಂಗ್‌ಫ್ಯೂಸ್ ಕ್ಲೈಂಟ್ ಅನ್ನು ಪ್ರಾರಂಭಿಸುತ್ತದೆ.


In [6]:
from langfuse import Langfuse
 
# Filter out Autogen OpenTelemetryspans
langfuse = Langfuse(
    blocked_instrumentation_scopes=["autogen SingleThreadedAgentRuntime"]
)
 
# Verify connection
if langfuse.auth_check():
    print("Langfuse client is authenticated and ready!")
else:
    print("Authentication failed. Please check your credentials and host.")

Langfuse client is authenticated and ready!


## ಹಂತ 2: OpenLit ಸಾಧನವನ್ನು ಪ್ರಾರಂಭಿಸಿ

ಈಗ, ನಾವು [OpenLit](https://github.com/openlit/openlit) ಸಾಧನವನ್ನು ಪ್ರಾರಂಭಿಸುತ್ತೇವೆ. OpenLit ಸ್ವಯಂಚಾಲಿತವಾಗಿ AutoGen ಕಾರ್ಯಾಚರಣೆಗಳನ್ನು ಸೆರೆಹಿಡಿದು OpenTelemetry (OTel) ಸ್ಪ್ಯಾನ್‌ಗಳನ್ನು Langfuse ಗೆ ರಫ್ತು ಮಾಡುತ್ತದೆ.


In [7]:
import openlit
 
# Initialize OpenLIT instrumentation. The disable_batch flag is set to true to process traces immediately.
openlit.init(tracer=langfuse._otel_tracer, disable_batch=True, disabled_instrumentors=["mistral"])

## ಹಂತ 3: ನಿಮ್ಮ ಏಜೆಂಟ್ ಅನ್ನು ಚಲಾಯಿಸಿ

ಈಗ ನಾವು ನಮ್ಮ ಸಾಧನ ಪರೀಕ್ಷಿಸಲು ಬಹು ತಿರುಗು ಏಜೆಂಟ್ ಅನ್ನು ಸ್ಥಾಪಿಸುತ್ತೇವೆ.


In [2]:
import os

from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_agentchat.base import TaskResult

from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat

In [3]:
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.environ["GITHUB_TOKEN"]),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
        "structured_output": False
    },
)

In [8]:
# 🍴 Agent 1 – proposes ONE healthy meal idea each turn
meal_planner_agent = AssistantAgent(
    "meal_planner_agent",
    model_client=client,
    description="A seasoned meal-planning coach who suggests balanced meals.",
    system_message="""
    You are a Meal-Planning Assistant with a decade of experience helping busy people prepare meals.
    Goal: propose the single best meal (breakfast, lunch, or dinner) given the user's context.
    Each response must contain ONLY one complete meal idea (title + very brief component list) — no extras.
    Keep it concise: skip greetings, chit-chat, and filler.
    """,
)

# 🥗 Agent 2 – checks nutritional quality & variety
nutritionist_agent = AssistantAgent(
    "nutritionist_agent",
    model_client=client,
    description="A registered dietitian ensuring meals meet nutritional standards.",
    system_message="""
    You are a Nutritionist focused on whole-food, macro-balanced eating.
    Evaluate the meal_planner_agent’s recommendation.
    If the meal is nutritionally sound, sufficiently varied, and portion-appropriate, respond with 'APPROVE'.
    Otherwise, give high-level guidance on how to improve it (e.g. 'add a plant-based protein') — do NOT provide a full alternative recipe.
    """,
)

In [9]:
# ✅ Chat stops once the nutritionist says APPROVE
termination = TextMentionTermination("APPROVE")

# 🔄 Alternate turns between the two agents until termination
team = RoundRobinGroupChat(
    [meal_planner_agent, nutritionist_agent],
    termination_condition=termination,
)

# Example kickoff
user_input = "I'm looking for a quick, delicious dinner I can prep after work. I have 30 minutes and minimal clean-up is ideal."

In [ ]:
with langfuse.start_as_current_span(name="create_meal_plan") as span:
    async for message in team.run_stream(task=user_input):
        if isinstance(message, TaskResult):
            print("Stop Reason:", message.stop_reason)
        else:
            print(message)

    span.update_trace(
        input=user_input,
        output=message.stop_reason,
    )

# Flush the trace to Langfuse for short-lived environments such as Jupyter Notebooks
langfuse.flush()

### ಟ್ರೇಸ್ ರಚನೆ

Langfuse ಒಂದು **ಟ್ರೇಸ್** ಅನ್ನು ದಾಖಲಿಸುತ್ತದೆ, ಇದು **ಸ್ಪ್ಯಾನ್ಸ್** ಅನ್ನು ಒಳಗೊಂಡಿರುತ್ತದೆ, ಅವು ನಿಮ್ಮ ಏಜೆಂಟ್‌ನ ತರ್ಕದ ಪ್ರತಿ ಹಂತವನ್ನು ಪ್ರತಿನಿಧಿಸುತ್ತವೆ. ಇಲ್ಲಿ, ಟ್ರೇಸ್ ಒಟ್ಟು ಏಜೆಂಟ್ ರನ್ ಮತ್ತು ಉಪ-ಸ್ಪ್ಯಾನ್ಸ್‌ಗಳನ್ನು ಒಳಗೊಂಡಿದೆ:
- ಊಟ ಯೋಜಕ ಏಜೆಂಟ್
- ಪೋಷಕತಜ್ಞರ ಏಜೆಂಟ್‌ಗಳು

ಇವುಗಳನ್ನು ಪರಿಶೀಲಿಸಿ ಸಮಯವನ್ನು ನಿಖರವಾಗಿ ಎಲ್ಲಿ ವ್ಯಯಿಸಲಾಗಿದೆ, ಎಷ್ಟು ಟೋಕನ್‌ಗಳನ್ನು ಬಳಸಲಾಗಿದೆ ಮತ್ತು ಇತ್ಯಾದಿ ತಿಳಿಯಲು:

![Langfuse ನಲ್ಲಿ ಟ್ರೇಸ್ ಟ್ರೀ](https://langfuse.com/images/cookbook/example-autogen-evaluation/trace-tree.png)

_[ಟ್ರೇಸ್‌ಗೆ ಲಿಂಕ್](https://cloud.langfuse.com/project/cloramnkj0002jz088vzn1ja4/traces/dac2b33e7cd709e685ccf86a137ecc64)_  


## ಆನ್‌ಲೈನ್ ಮೌಲ್ಯಮಾಪನ

ಆನ್‌ಲೈನ್ ಮೌಲ್ಯಮಾಪನವು ಏಜೆಂಟ್ ಅನ್ನು ನೈಜ, ವಾಸ್ತವಿಕ ಪರಿಸರದಲ್ಲಿ, ಅಂದರೆ ಉತ್ಪಾದನೆಯಲ್ಲಿ ನಿಜವಾದ ಬಳಕೆಯ ಸಮಯದಲ್ಲಿ ಮೌಲ್ಯಮಾಪನ ಮಾಡುವುದನ್ನು ಸೂಚಿಸುತ್ತದೆ. ಇದು ನಿಜವಾದ ಬಳಕೆದಾರರ ಸಂವಹನಗಳ ಮೇಲೆ ಏಜೆಂಟ್‌ನ ಕಾರ್ಯಕ್ಷಮತೆಯನ್ನು ನಿಗಾ ವಹಿಸುವುದು ಮತ್ತು ಫಲಿತಾಂಶಗಳನ್ನು ನಿರಂತರವಾಗಿ ವಿಶ್ಲೇಷಿಸುವುದನ್ನು ಒಳಗೊಂಡಿದೆ.

### ಉತ್ಪಾದನೆಯಲ್ಲಿ ಟ್ರ್ಯಾಕ್ ಮಾಡಲು ಸಾಮಾನ್ಯ ಮೆಟ್ರಿಕ್‌ಗಳು

1. **ವೆಚ್ಚಗಳು** — ಸಾಧನವು ಟೋಕನ್ ಬಳಕೆಯನ್ನು ಸೆರೆಹಿಡಿಯುತ್ತದೆ, ಇದನ್ನು ಪ್ರತಿ ಟೋಕನ್‌ಗೆ ಬೆಲೆಯನ್ನು ನೀಡುವ ಮೂಲಕ ಅಂದಾಜು ವೆಚ್ಚಗಳಲ್ಲಿ ಪರಿವರ್ತಿಸಬಹುದು.
2. **ವಿಲಂಬ** — ಪ್ರತಿ ಹಂತವನ್ನು ಪೂರ್ಣಗೊಳಿಸಲು ತೆಗೆದುಕೊಳ್ಳುವ ಸಮಯವನ್ನು ಅಥವಾ ಸಂಪೂರ್ಣ ಕಾರ್ಯಾಚರಣೆಯನ್ನು ಗಮನಿಸಿ.
3. **ಬಳಕೆದಾರ ಪ್ರತಿಕ್ರಿಯೆ** — ಬಳಕೆದಾರರು ನೇರ ಪ್ರತಿಕ್ರಿಯೆಯನ್ನು (ತುಂಬ್ಸ್ ಅಪ್/ಡೌನ್) ನೀಡಬಹುದು, ಇದು ಏಜೆಂಟ್ ಅನ್ನು ಸುಧಾರಿಸಲು ಅಥವಾ ಸರಿಪಡಿಸಲು ಸಹಾಯ ಮಾಡುತ್ತದೆ.
4. **LLM-ಆಸ್-ಎ-ಜಡ್ಜ್** — ನಿಮ್ಮ ಏಜೆಂಟ್‌ನ ಔಟ್‌ಪುಟ್ ಅನ್ನು ನಿಕಟ ನೈಜ-ಸಮಯದಲ್ಲಿ (ಉದಾ., ವಿಷಾಕ್ತತೆ ಅಥವಾ ಸರಿಯಾದತೆಯನ್ನು ಪರಿಶೀಲಿಸುವುದು) ಮೌಲ್ಯಮಾಪನ ಮಾಡಲು ಪ್ರತ್ಯೇಕ LLM ಅನ್ನು ಬಳಸಿರಿ.

ಕೆಳಗಿನವು ಈ ಮೆಟ್ರಿಕ್‌ಗಳ ಉದಾಹರಣೆಗಳನ್ನು ತೋರಿಸುತ್ತದೆ.


#### 1. ವೆಚ್ಚಗಳು

ಕೆಳಗಿನ ಚಿತ್ರದಲ್ಲಿ `gpt-4o-mini` ಕರೆಗಳ ಬಳಕೆಯನ್ನು ತೋರಿಸಲಾಗಿದೆ. ಇದು ವೆಚ್ಚದಾಯಕ ಹಂತಗಳನ್ನು ಗುರುತಿಸಲು ಮತ್ತು ನಿಮ್ಮ ಏಜೆಂಟ್ ಅನ್ನು ಆಪ್ಟಿಮೈಸ್ ಮಾಡಲು ಸಹಾಯಕವಾಗಿದೆ.

![ವೆಚ್ಚಗಳು](https://langfuse.com/images/cookbook/example-autogen-evaluation/gpt-4o-costs.png) 

_[ಟ್ರೇಸ್‌ಗೆ ಲಿಂಕ್](https://cloud.langfuse.com/project/cloramnkj0002jz088vzn1ja4/traces/dac2b33e7cd709e685ccf86a137ecc64)_


#### 2. ವಿಳಂಬ

ಪ್ರತಿ ಹಂತವನ್ನು ಪೂರ್ಣಗೊಳಿಸಲು ಎಷ್ಟು ಸಮಯ ತೆಗೆದುಕೊಂಡಿತು ಎಂಬುದನ್ನು ನಾವು ನೋಡಬಹುದು. ಕೆಳಗಿನ ಉದಾಹರಣೆಯಲ್ಲಿ, ಸಂಪೂರ್ಣ ಕಾರ್ಯಾಚರಣೆ ಸುಮಾರು 3 ಸೆಕೆಂಡುಗಳನ್ನು ತೆಗೆದುಕೊಂಡಿತು, ಇದನ್ನು ಹಂತಗಳ ಪ್ರಕಾರ ವಿಭಜಿಸಬಹುದು. ಇದು ನಿಮಗೆ bottlenecks ಅನ್ನು ಗುರುತಿಸಲು ಮತ್ತು ನಿಮ್ಮ ಏಜೆಂಟ್ ಅನ್ನು ಆಪ್ಟಿಮೈಸ್ ಮಾಡಲು ಸಹಾಯ ಮಾಡುತ್ತದೆ.

![ವಿಳಂಬ](https://langfuse.com/images/cookbook/example-autogen-evaluation/agent-latency.png) 

_[ಟ್ರೇಸ್‌ಗೆ ಲಿಂಕ್](https://cloud.langfuse.com/project/cloramnkj0002jz088vzn1ja4/traces/dac2b33e7cd709e685ccf86a137ecc64?display=timeline)_


#### 3. ಬಳಕೆದಾರರ ಪ್ರತಿಕ್ರಿಯೆ

ನಿಮ್ಮ ಏಜೆಂಟ್ ಬಳಕೆದಾರ ಇಂಟರ್ಫೇಸ್ನಲ್ಲಿ ಅಳವಡಿಸಲಾಗಿದ್ದರೆ, ನೀವು ನೇರ ಬಳಕೆದಾರರ ಪ್ರತಿಕ್ರಿಯೆಯನ್ನು (ಚಾಟ್ UIಯಲ್ಲಿ ತುಂಬ್ಸ್-ಅಪ್/ಡೌನ್ ಹೋಲುವಂತೆ) ದಾಖಲಿಸಬಹುದು.


In [10]:
from langfuse import get_client
 
langfuse = get_client()
 
# Option 1: Use the yielded span object from the context manager
with langfuse.start_as_current_span(
    name="autogen-request-user-feedback-1") as span:
    
    async for message in team.run_stream(task="Create a meal with potatoes"):
            if isinstance(message, TaskResult):
                print("Stop Reason:", message.stop_reason)
            else:
                print(message)    
 
    # Score using the span object
    span.score_trace(
        name="user-feedback",
        value=1,
        data_type="NUMERIC",
        comment="This was delicious, thank you"
    )
 
# Option 2: Use langfuse.score_current_trace() if still in context
with langfuse.start_as_current_span(name="autogen-request-user-feedback-2") as span:
    # ... Autogen execution ...

    async for message in team.run_stream(task="I am allergic to gluten."):
            if isinstance(message, TaskResult):
                print("Stop Reason:", message.stop_reason)
            else:
                print(message)    
 
    # Score using current context
    langfuse.score_current_trace(
        name="user-feedback",
        value=1,
        data_type="NUMERIC"
    )

id='da068880-22ae-4f01-9f01-2bb231939089' source='user' models_usage=None metadata={} created_at=datetime.datetime(2025, 7, 2, 16, 20, 43, 732669, tzinfo=datetime.timezone.utc) content='Create a meal with potatoes' type='TextMessage'
id='ad937ce4-3534-493f-824b-ca9c226b5287' source='meal_planner_agent' models_usage=RequestUsage(prompt_tokens=95, completion_tokens=30) metadata={} created_at=datetime.datetime(2025, 7, 2, 16, 20, 45, 186423, tzinfo=datetime.timezone.utc) content='Potato and Spinach Frittata  \n- Eggs  \n- Potatoes  \n- Fresh spinach  \n- Onion  \n- Cheese (optional)  ' type='TextMessage'
id='50fd33c1-057f-49fe-afad-ee86d164296d' source='nutritionist_agent' models_usage=RequestUsage(prompt_tokens=132, completion_tokens=4) metadata={} created_at=datetime.datetime(2025, 7, 2, 16, 20, 45, 581059, tzinfo=datetime.timezone.utc) content='APPROVE' type='TextMessage'
Stop Reason: Text 'APPROVE' mentioned
id='e371de6c-e5fc-42c1-8eda-e5b8cd5accab' source='user' models_usage=None met

In [ ]:
# Option 3: Use create_score() with trace ID (when outside context)
langfuse.create_score(
    trace_id="predefined_trace_id",
    name="user-feedback",
    value=1,
    data_type="NUMERIC",
    comment="This was correct, thank you"
)

ಬಳಕೆದಾರರ ಪ್ರತಿಕ್ರಿಯೆಯನ್ನು ನಂತರ Langfuse ನಲ್ಲಿ ಸೆರೆಹಿಡಿಯಲಾಗುತ್ತದೆ:

![ಬಳಕೆದಾರರ ಪ್ರತಿಕ್ರಿಯೆಯನ್ನು Langfuse ನಲ್ಲಿ ಸೆರೆಹಿಡಿಯಲಾಗುತ್ತಿದೆ](https://langfuse.com/images/cookbook/example-autogen-evaluation/user-feedback.png)


#### 4. ಸ್ವಯಂಚಾಲಿತ LLM-as-a-Judge ಅಂಕಿ ಮೌಲ್ಯನಿರ್ಣಯ

LLM-as-a-Judge ನಿಮ್ಮ ಏಜೆಂಟ್‌ನ ಔಟ್‌ಪುಟ್ ಅನ್ನು ಸ್ವಯಂಚಾಲಿತವಾಗಿ ಮೌಲ್ಯಮಾಪನ ಮಾಡುವ ಮತ್ತೊಂದು ವಿಧಾನವಾಗಿದೆ. ಔಟ್‌ಪುಟ್‌ನ ಸರಿಯಾದತೆ, ವಿಷಕಾರಿ ಸ್ವಭಾವ, ಶೈಲಿ ಅಥವಾ ನೀವು ಗಮನಿಸುವ ಯಾವುದೇ ಮಾನದಂಡವನ್ನು ಅಂದಾಜಿಸಲು ಪ್ರತ್ಯೇಕ LLM ಕರೆಗಳನ್ನು ಹೊಂದಿಸಬಹುದು.

**ಕಾರ್ಯಪ್ರವಾಹ**:
1. ನೀವು **ಮೌಲ್ಯಮಾಪನ ಟೆಂಪ್ಲೇಟು** ಅನ್ನು ವ್ಯಾಖ್ಯಾನಿಸುತ್ತೀರಿ, ಉದಾಹರಣೆಗೆ, "ಪಠ್ಯವು ವಿಷಕಾರಿ ಆಗಿದೆಯೇ ಎಂದು ಪರಿಶೀಲಿಸಿ."
2. ನೀವು ನ್ಯಾಯಾಧೀಶ-ಮಾದರಿಯಾಗಿ ಬಳಸುವ ಮಾದರಿಯನ್ನು ಹೊಂದಿಸುತ್ತೀರಿ; ಈ ಸಂದರ್ಭದಲ್ಲಿ `gpt-4o-mini` ಅನ್ನು Azure ಮೂಲಕ ಕೇಳಲಾಗುತ್ತದೆ.
2. ಪ್ರತಿ ಬಾರಿ ನಿಮ್ಮ ಏಜೆಂಟ್ ಔಟ್‌ಪುಟ್ ಅನ್ನು ರಚಿಸಿದಾಗ, ನೀವು ಆ ಔಟ್‌ಪುಟ್ ಅನ್ನು ಟೆಂಪ್ಲೇಟಿನೊಂದಿಗೆ "ನ್ಯಾಯಾಧೀಶ" LLM ಗೆ ಪಾಸ್ ಮಾಡುತ್ತೀರಿ.
3. ನ್ಯಾಯಾಧೀಶ LLM ಪ್ರತಿಕ್ರಿಯೆ ನೀಡುತ್ತದೆ ಅಂಕಿ ಅಥವಾ ಲೇಬಲ್‌ನೊಂದಿಗೆ, ಇದನ್ನು ನೀವು ನಿಮ್ಮ ಅವಲೋಕನ ಸಾಧನದಲ್ಲಿ ಲಾಗ್ ಮಾಡುತ್ತೀರಿ.

Langfuse ನಿಂದ ಉದಾಹರಣೆ:

![LLM-as-a-Judge ಮೌಲ್ಯಮಾಪಕ](https://langfuse.com/images/cookbook/example-autogen-evaluation/evaluator.png)


In [12]:
with langfuse.start_as_current_span(name="autogen-request-user-feedback-2") as span:

    async for message in team.run_stream(task="I am a picky eater and not sure if you find something for me."):
            if isinstance(message, TaskResult):
                print("Stop Reason:", message.stop_reason)
            else:
                print(message) 

    span.update_trace(
        input=user_input,
        output=message.stop_reason,
    )

langfuse.flush()

id='eefc628d-502f-451a-8f70-be486f62f8c5' source='user' models_usage=None metadata={} created_at=datetime.datetime(2025, 7, 2, 16, 38, 29, 171393, tzinfo=datetime.timezone.utc) content='I am a picky eater and not sure if you find something for me.' type='TextMessage'
id='13b3e14b-bcf7-42a5-80d6-54b0c7be765e' source='meal_planner_agent' models_usage=RequestUsage(prompt_tokens=352, completion_tokens=27) metadata={} created_at=datetime.datetime(2025, 7, 2, 16, 38, 30, 433516, tzinfo=datetime.timezone.utc) content='Chicken Alfredo Pasta  \n- Gluten-free pasta  \n- Grilled chicken breast  \n- Heavy cream  \n- Parmesan cheese  \n- Garlic  ' type='TextMessage'
id='550f2dee-0e08-4bbd-b67f-991b467328f1' source='nutritionist_agent' models_usage=RequestUsage(prompt_tokens=386, completion_tokens=17) metadata={} created_at=datetime.datetime(2025, 7, 2, 16, 38, 31, 505173, tzinfo=datetime.timezone.utc) content='Consider incorporating some vegetables, like spinach or broccoli, to increase the nutrien

ನೀವು ಈ ಉದಾಹರಣೆಯ ಉತ್ತರವನ್ನು "ವಿಷಮಯವಿಲ್ಲದ" ಎಂದು ತೀರ್ಮಾನಿಸಲಾಗಿದೆ ಎಂದು ನೋಡಬಹುದು.

![LLM-as-a-Judge ಮೌಲ್ಯಮಾಪನ ಅಂಕೆ](https://langfuse.com/images/cookbook/example-autogen-evaluation/llm-as-a-judge-score.png)


#### 5. ಗಮನಾರ್ಹತೆ ಮೆಟ್ರಿಕ್‌ಗಳ ಅವಲೋಕನ

ಈ ಎಲ್ಲಾ ಮೆಟ್ರಿಕ್‌ಗಳನ್ನು ಡ್ಯಾಶ್‌ಬೋರ್ಡ್‌ಗಳಲ್ಲಿ ಒಟ್ಟಿಗೆ ವೀಕ್ಷಿಸಬಹುದು. ಇದು ನಿಮ್ಮ ಏಜೆಂಟ್ ಹಲವಾರು ಸೆಷನ್‌ಗಳಲ್ಲಿ ಹೇಗೆ ಕಾರ್ಯನಿರ್ವಹಿಸುತ್ತಿದೆ ಎಂಬುದನ್ನು ತ್ವರಿತವಾಗಿ ನೋಡಲು ಮತ್ತು ಸಮಯದೊಂದಿಗೆ ಗುಣಮಟ್ಟದ ಮೆಟ್ರಿಕ್‌ಗಳನ್ನು ಟ್ರ್ಯಾಕ್ ಮಾಡಲು ನಿಮಗೆ ಸಹಾಯ ಮಾಡುತ್ತದೆ.

![ಗಮನಾರ್ಹತೆ ಮೆಟ್ರಿಕ್‌ಗಳ ಅವಲೋಕನ](https://langfuse.com/images/cookbook/example-autogen-evaluation/dashboard.png)


## ಆಫ್‌ಲೈನ್ ಮೌಲ್ಯಮಾಪನ

ಆನ್‌ಲೈನ್ ಮೌಲ್ಯಮಾಪನವು ತಕ್ಷಣದ ಪ್ರತಿಕ್ರಿಯೆಗಾಗಿ ಅಗತ್ಯವಿದೆ, ಆದರೆ ನೀವು **ಆಫ್‌ಲೈನ್ ಮೌಲ್ಯಮಾಪನ** ಕೂಡ ಮಾಡಬೇಕು—ಅರ್ಥಾತ್ ಅಭಿವೃದ್ಧಿಯ ಮೊದಲು ಅಥವಾ ಅವಧಿಯಲ್ಲಿ ವ್ಯವಸ್ಥಿತ ತಪಾಸಣೆಗಳು. ಇದು ಬದಲಾವಣೆಗಳನ್ನು ಉತ್ಪಾದನೆಯಲ್ಲಿ ಜಾರಿಗೆ ತರಲು ಮುಂಚೆ ಗುಣಮಟ್ಟ ಮತ್ತು ನಂಬಿಕೆಪಾತ್ರತೆಯನ್ನು ಕಾಪಾಡಲು ಸಹಾಯ ಮಾಡುತ್ತದೆ.


### ಡೇಟಾಸೆಟ್ ಮೌಲ್ಯಮಾಪನ

ಆಫ್‌ಲೈನ್ ಮೌಲ್ಯಮಾಪನದಲ್ಲಿ, ನೀವು ಸಾಮಾನ್ಯವಾಗಿ:
1. ಪ್ರಾಮಾಣಿಕ ಡೇಟಾಸೆಟ್ ಹೊಂದಿರುತ್ತೀರಿ (ಪ್ರಶ್ನೆ ಮತ್ತು ನಿರೀಕ್ಷಿತ ಉತ್ತರ ಜೋಡಿಗಳೊಂದಿಗೆ)
2. ಆ ಡೇಟಾಸೆಟ್‌ನಲ್ಲಿ ನಿಮ್ಮ ಏಜೆಂಟ್ ಅನ್ನು ಚಲಾಯಿಸುತ್ತೀರಿ
3. ಫಲಿತಾಂಶಗಳನ್ನು ನಿರೀಕ್ಷಿತ ಫಲಿತಾಂಶಗಳೊಂದಿಗೆ ಹೋಲಿಸುತ್ತೀರಿ ಅಥವಾ ಹೆಚ್ಚುವರಿ ಅಂಕಗಳ ನಿರ್ಣಯ ವಿಧಾನವನ್ನು ಬಳಸುತ್ತೀರಿ

ಕೆಳಗೆ, ನಾವು [q&a-dataset](https://huggingface.co/datasets/junzhang1207/search-dataset) ಬಳಸಿ ಈ ವಿಧಾನವನ್ನು ತೋರಿಸುತ್ತೇವೆ, ಇದು ಪ್ರಶ್ನೆಗಳು ಮತ್ತು ನಿರೀಕ್ಷಿತ ಉತ್ತರಗಳನ್ನು ಒಳಗೊಂಡಿರುತ್ತದೆ.


In [16]:
import pandas as pd
from datasets import load_dataset
 
# Fetch search-dataset from Hugging Face
dataset = load_dataset("junzhang1207/search-dataset", split = "train")
df = pd.DataFrame(dataset)
print("First few rows of search-dataset:")
print(df.head())

/Users/jannik/Documents/GitHub/ai-agents-for-beginners/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


First few rows of search-dataset:
                                     id  \
0  20caf138-0c81-4ef9-be60-fe919e0d68d4   
1  1f37d9fd-1bcc-4f79-b004-bc0e1e944033   
2  76173a7f-d645-4e3e-8e0d-cca139e00ebe   
3  5f5ef4ca-91fe-4610-a8a9-e15b12e3c803   
4  64dbed0d-d91b-4acd-9a9c-0a7aa83115ec   

                                            question  \
0                 steve jobs statue location budapst   
1  Why is the Battle of Stalingrad considered a t...   
2  In what year did 'The Birth of a Nation' surpa...   
3  How many Russian soldiers surrendered to AFU i...   
4   What event led to the creation of Google Images?   

                                     expected_answer       category       area  
0  The Steve Jobs statue is located in Budapest, ...           Arts  Knowledge  
1  The Battle of Stalingrad is considered a turni...   General News       News  
2  This question is based on a false premise. 'Th...  Entertainment       News  
3  About 300 Russian soldiers surrendered to t

ಮುಂದೆ, ನಾವು Langfuse ನಲ್ಲಿ ಡೇಟಾಸೆಟ್ ಎಂಟಿಟಿಯನ್ನು ರನ್‌ಗಳನ್ನು ಟ್ರ್ಯಾಕ್ ಮಾಡಲು ಸೃಷ್ಟಿಸುತ್ತೇವೆ. ನಂತರ, ಡೇಟಾಸೆಟ್‌ನ ಪ್ರತಿ ಐಟಂ ಅನ್ನು ವ್ಯವಸ್ಥೆಗೆ ಸೇರಿಸುತ್ತೇವೆ.


In [17]:
from langfuse import Langfuse
langfuse = Langfuse()
 
langfuse_dataset_name = "qa-dataset_autogen-agent"
 
# Create a dataset in Langfuse
langfuse.create_dataset(
    name=langfuse_dataset_name,
    description="q&a dataset uploaded from Hugging Face",
    metadata={
        "date": "2025-03-21",
        "type": "benchmark"
    }
)

Dataset(id='cmcm7524d00kjad07s2cjwqcf', name='qa-dataset_autogen-agent', description='q&a dataset uploaded from Hugging Face', metadata={'date': '2025-03-21', 'type': 'benchmark'}, project_id='cloramnkj0002jz088vzn1ja4', created_at=datetime.datetime(2025, 7, 2, 16, 54, 7, 357000, tzinfo=datetime.timezone.utc), updated_at=datetime.datetime(2025, 7, 2, 16, 54, 7, 357000, tzinfo=datetime.timezone.utc))

In [18]:
df_25 = df.sample(25) # For this example, we upload only 25 dataset questions

for idx, row in df_25.iterrows():
    langfuse.create_dataset_item(
        dataset_name=langfuse_dataset_name,
        input={"text": row["question"]},
        expected_output={"text": row["expected_answer"]}
    )

![Dataset items in Langfuse](https://langfuse.com/images/cookbook/example-autogen-evaluation/example-dataset.png)


#### ಡೇಟಾಸೆಟ್‌ನಲ್ಲಿ ಏಜೆಂಟ್ ಅನ್ನು ಚಲಿಸುವುದು

ಮೊದಲು, ನಾವು ಸರಳ Autogen ಏಜೆಂಟ್ ಅನ್ನು ಸೇರಿಸುತ್ತೇವೆ, ಇದು Azure OpenAI ಮಾದರಿಗಳನ್ನು ಬಳಸಿಕೊಂಡು ಪ್ರಶ್ನೆಗಳಿಗೆ ಉತ್ತರಿಸುತ್ತದೆ.


In [8]:
import os
from dotenv import load_dotenv

from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken
from autogen_agentchat.messages import TextMessage

In [ ]:
load_dotenv()
client = AzureAIChatCompletionClient(
    model="gpt-4o",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.getenv("GITHUB_TOKEN")),
    max_tokens=5000,
    model_info={
        "json_output": True,
        "function_calling": False,
        "vision": False,
        "family": "unknown",
        "structured_output": True,
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

In [18]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[],
    system_message="You are participant in a quizz show and you are given a question. You need to create a short answer to the question.",
)

ನಂತರ, ನಾವು `my_agent()` ಎಂಬ ಸಹಾಯಕ ಕಾರ್ಯವನ್ನು ವ್ಯಾಖ್ಯಾನಿಸುತ್ತೇವೆ.


In [19]:
async def my_agent(user_query: str):

    with langfuse.start_as_current_span(name="autogen-trace") as span:

        # Execute the agent response
        response = await agent.on_messages(
            [TextMessage(content=user_query, source="user")],
            cancellation_token=CancellationToken(),
        )

        span.update_trace(
            input=user_query,
            output=response.chat_message.content,
        )

    return str(response.chat_message.content)

# Test the function
await my_agent("What is the capital of France?")

'The capital of France is Paris.'

ಅಂತಿಮವಾಗಿ, ನಾವು ಪ್ರತಿ ಡೇಟಾಸೆಟ್ ಐಟಂ ಮೇಲೆ ಲೂಪ್ ಮಾಡುತ್ತೇವೆ, ಏಜೆಂಟ್ ಅನ್ನು ರನ್ ಮಾಡುತ್ತೇವೆ ಮತ್ತು ಟ್ರೇಸ್ ಅನ್ನು ಡೇಟಾಸೆಟ್ ಐಟಂಗೆ ಲಿಂಕ್ ಮಾಡುತ್ತೇವೆ. ಅಗತ್ಯವಿದ್ದರೆ ನಾವು ವೇಗವಾದ ಮೌಲ್ಯಮಾಪನ ಅಂಕೆಯನ್ನು ಕೂಡ ಅಟ್ಯಾಚ್ ಮಾಡಬಹುದು.


In [20]:
dataset_name = "qa-dataset_autogen-agent"
current_run_name = "dev_tasks_run-autogen_gpt-4.1" # Identifies this specific evaluation run
current_run_metadata={"model_provider": "Azure", "model": "gpt-4.1"}
current_run_description="Evaluation run for Autogen model on July 3rd"

dataset = langfuse.get_dataset('qa-dataset_autogen-agent')

for item in dataset.items:
    print(f"Running evaluation for item: {item.id} (Input: {item.input})")
 
    # Use the item.run() context manager
    with item.run(
        run_name=current_run_name,
        run_metadata=current_run_metadata,
        run_description=current_run_description
    ) as root_span: 
        # All subsequent langfuse operations within this block are part of this trace.
        generated_answer = await my_agent(user_query = item.input["text"])
    
    print("Generated Answer: ", generated_answer)
 
print(f"\nFinished processing dataset '{dataset_name}' for run '{current_run_name}'.")

langfuse.flush()

Running evaluation for item: 09810cc4-9992-4712-a3b2-7224da31776a (Input: {'text': 'In Hindu mythology, which deity is the Ganges river dolphin associated with?'})
Generated Answer:  In Hindu mythology, the Ganges river dolphin is associated with the deity Ganga.
Running evaluation for item: bb113f94-7723-47c6-8c34-59d883044514 (Input: {'text': 'What significant discovery did the LHCb collaboration report in 2015?'})
Generated Answer:  In 2015, the LHCb collaboration reported the discovery of pentaquark particles.
Running evaluation for item: 4d8ae54e-ceab-46d0-ad2c-6e8e223589a9 (Input: {'text': 'What is the MÄ\x81ori name for the red-crowned parakeet?'})
Generated Answer:  The Māori name for the red-crowned parakeet is kākāriki.
Running evaluation for item: 21e5a0d5-f619-4a73-868e-9955053b3e72 (Input: {'text': 'Who starred in the 1978 television film adaptation of Les MisÃ©rables?'})
Generated Answer:  Richard Jordan starred as Jean Valjean in the 1978 television film adaptation of Le

ನೀವು ಈ ಪ್ರಕ್ರಿಯೆಯನ್ನು ವಿಭಿನ್ನ ಏಜೆಂಟ್ ಸಂರಚನೆಗಳೊಂದಿಗೆ ಪುನರಾವರ್ತಿಸಬಹುದು, ಉದಾಹರಣೆಗೆ:  
- ಮಾದರಿಗಳು (gpt-4o-mini, gpt-4.1, ಇತ್ಯಾದಿ)  
- ಪ್ರಾಂಪ್ಟ್‌ಗಳು  
- ಸಾಧನಗಳು (ಹುಡುಕು vs. ಹುಡುಕಾಟವಿಲ್ಲದೆ)  
- ಏಜೆಂಟ್‌ನ ಸಂಕೀರ್ಣತೆ (ಬಹು ಏಜೆಂಟ್ vs. ಏಕ ಏಜೆಂಟ್)  

ನಂತರ ಅವುಗಳನ್ನು Langfuse ನಲ್ಲಿ ಪಕ್ಕಪಕ್ಕದಲ್ಲಿ ಹೋಲಿಸಿ. ಈ ಉದಾಹರಣೆಯಲ್ಲಿ, ನಾನು 25 ಡೇಟಾಸೆಟ್ ಪ್ರಶ್ನೆಗಳ ಮೇಲೆ ಏಜೆಂಟ್ ಅನ್ನು 3 ಬಾರಿ ಚಲಾಯಿಸಿದೆ. ಪ್ರತಿ ಬಾರಿ, ನಾನು ವಿಭಿನ್ನ Azure OpenAI ಮಾದರಿಯನ್ನು ಬಳಸಿದೆ. ನೀವು ದೊಡ್ಡ ಮಾದರಿಯನ್ನು ಬಳಸಿದಾಗ (ನಿರೀಕ್ಷಿತಂತೆ) ಸರಿಯಾಗಿ ಉತ್ತರಿಸಿದ ಪ್ರಶ್ನೆಗಳ ಪ್ರಮಾಣವು ಸುಧಾರಿಸುತ್ತದೆ ಎಂದು ನೋಡಬಹುದು. `correct_answer` ಅಂಕೆಯನ್ನು [LLM-as-a-Judge Evaluator](https://langfuse.com/docs/scores/model-based-evals) ಮೂಲಕ ರಚಿಸಲಾಗಿದೆ, ಇದು ಡೇಟಾಸೆಟ್‌ನಲ್ಲಿ ನೀಡಿದ ಮಾದರಿ ಉತ್ತರದ ಆಧಾರದ ಮೇಲೆ ಪ್ರಶ್ನೆಯ ಸರಿಯಾದತೆಯನ್ನು ತೀರ್ಮಾನಿಸಲು ಹೊಂದಿಸಲಾಗಿದೆ.  

![ಡೇಟಾಸೆಟ್ ರನ್ ಅವಲೋಕನ](https://langfuse.com/images/cookbook/example-autogen-evaluation/dataset_runs.png)  
![ಡೇಟಾಸೆಟ್ ರನ್ ಹೋಲಿಕೆ](https://langfuse.com/images/cookbook/example-autogen-evaluation/dataset-run-comparison.png)  


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**ಅಸ್ವೀಕಾರ**:  
ಈ ದಾಖಲೆ AI ಅನುವಾದ ಸೇವೆ [Co-op Translator](https://github.com/Azure/co-op-translator) ಬಳಸಿ ಅನುವಾದಿಸಲಾಗಿದೆ. ನಾವು ನಿಖರತೆಯನ್ನು ಸಾಧಿಸಲು ಪ್ರಯತ್ನಿಸುತ್ತಿದ್ದರೂ, ದಯವಿಟ್ಟು ಗಮನಿಸಿ, ಸ್ವಯಂಚಾಲಿತ ಅನುವಾದಗಳಲ್ಲಿ ದೋಷಗಳು ಅಥವಾ ಅಸಮರ್ಪಕತೆಗಳು ಇರಬಹುದು. ಮೂಲ ಭಾಷೆಯಲ್ಲಿರುವ ಮೂಲ ದಾಖಲೆ ಪ್ರಾಮಾಣಿಕ ಮೂಲವೆಂದು ಪರಿಗಣಿಸಬೇಕು. ಪ್ರಮುಖ ಮಾಹಿತಿಗಾಗಿ, ವೃತ್ತಿಪರ ಮಾನವ ಅನುವಾದವನ್ನು ಶಿಫಾರಸು ಮಾಡಲಾಗುತ್ತದೆ. ಈ ಅನುವಾದದ ಬಳಕೆಯಿಂದ ಉಂಟಾಗುವ ಯಾವುದೇ ತಪ್ಪುಅರ್ಥಗಳು ಅಥವಾ ತಪ್ಪುಅರ್ಥೈಸುವಿಕೆಗೆ ನಾವು ಹೊಣೆಗಾರರಲ್ಲ.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
